<a href="https://colab.research.google.com/github/Abdullah-Khan-1992/Piaic/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
%pip install -qU langchain-pinecone pinecone-notebooks

In [14]:
from google.colab import userdata

from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

In [15]:
import time

index_name = "online-rag-project01"  # change if desired

pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

index = pc.Index(index_name)

In [17]:
%pip install --upgrade --quiet  langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.4 MB/s eta 0:00:00


In [18]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [19]:
vector = embeddings.embed_query("we are building a RAG text")

In [21]:
vector[:5]

[0.041496895253658295,
 -0.04191027954220772,
 -0.05892135202884674,
 -0.008314159698784351,
 0.00635612104088068]

In [22]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [24]:
# Data save

from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

# Data Ratrieve

In [25]:
len(documents)

10

In [27]:
from uuid import uuid4
uuid4()

UUID('9fdcb1d8-1663-4775-a7ed-ba7f83b3f545')

In [28]:
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['769b3c2e-256a-443b-8732-844a9323ebf1',
 'bc7c8b2d-0fe0-4658-a9b3-664786b9faa7',
 'c6b55dbb-0842-42a3-8727-b790f78382d3',
 '13a229a4-4be8-4e41-aac4-dda7eac16d29',
 '8cf41733-4087-4bdf-ade6-1d6a54fdd181',
 '7c780d48-7c4b-4a4d-86b0-97eafe98a9f7',
 'bed58c69-6048-4b9e-8549-97aeee97d7a7',
 '1d242856-1b4e-4a9a-9d31-517c4eae34d3',
 '187b44ff-7d6c-4710-8b6e-b4e229bf3245',
 'acc6d437-a509-4f50-b4fa-f650aaf0b91f']

In [29]:
# Data Ratrieve

results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]


In [30]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.667716] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


In [32]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [36]:
def answer_to_user(query: str):

  # Vector search
  vector_results = vector_store.similarity_search(query, k=2)
  print(len(vector_results))

  # pass to model vector results + user query

  final_answer = llm.invoke(f"ANSWER THIS USER QUERY:{query}, here are some refrences to answer {vector_results}")

  return final_answer


In [37]:
answer = answer_to_user("Langchain provides abstractions to make working with LLMs easy.")

2


In [38]:
answer.content

"The provided text focuses on LangGraph, not Langchain.  Therefore, it doesn't answer the user query about Langchain's abstractions for simplifying LLM work.  The documents only state that LangGraph is a good framework for building certain types of applications.  There is no relevant information about Langchain in the given context.\n"